In [1]:
%%bash -c "cat > diagram.h"
#pragma once

#include <vector>
#include <algorithm>
#include <cmath>

struct TSector {
    std::vector<std::pair<float, int>> Objects;
};

struct TDiagram {
    std::vector<TSector> Sectors;
};

inline float DiagramDistance(const TDiagram& a, const TDiagram& b) {
    if (a.Sectors.size() != b.Sectors.size()) {
        return -1;
    }
    float resultDistance = 1e10;
    for (int shift = 0; shift < (int)a.Sectors.size(); ++shift) {
        float curDistance = 0;
        for (int i = 0; i < (int)a.Sectors.size(); ++i) {
            int bi = (i + shift) % b.Sectors.size();
            int minObjCount = std::min(a.Sectors[i].Objects.size(), 
                                       b.Sectors[bi].Objects.size());
            int maxObjCount = std::max(a.Sectors[i].Objects.size(), 
                                       b.Sectors[bi].Objects.size());
            for (int j = 0; j < minObjCount; ++j) {
                if (a.Sectors[i].Objects[j].second != b.Sectors[bi].Objects[j].second) {
                    curDistance += 1.0 / ((j + 1) * (j + 1));
                }
            }
            for (int j = minObjCount; j < maxObjCount; ++j) {
                curDistance += 1.0 / ((j + 1) * (j + 1));
            }
            
        }
        resultDistance = std::min(resultDistance, curDistance);
    }
    return resultDistance;
}

In [2]:
%%script bash -c "cat > diagram.pxd"
from libcpp.vector cimport vector
from libcpp.pair cimport pair

cdef extern from "diagram.h" nogil:
    cdef cppclass TSector:
        TSector()
        vector[pair[float, int]] Objects
    
    cdef cppclass TDiagram:
        TDiagram()
        vector[TSector] Sectors
        
    float DiagramDistance(const TDiagram& a, const TDiagram& b)
    
cdef class Diagram:
    cdef TDiagram diagram
    cdef init_cpp_from_python(self, diagram_obj)

In [3]:
%%script bash -c "cat > diagram.pyx"
# distutils: language=c++
from libcpp.vector cimport vector
from libcpp.pair cimport pair

OBJ_TYPES = ["water", "wood"]
OBJ_TYPE_TO_NUM = {t: i for i, t in enumerate(OBJ_TYPES)}

cdef class Diagram:
    '''
        Optimized for fast dist_to operation
        Constructing is relatively slow operation 
        
    '''
    
    def __init__(self, diagram_obj):
        """
            Example: 'Diagram([[(1, "water")], [(1, "forest"), (2, "water")]])'
            Support types: OBJ_TYPES
        """ 
        self.init_cpp_from_python(diagram_obj)
        
    cdef init_cpp_from_python(self, diagram_obj):
        cdef TSector cpp_s
        for s in diagram_obj:  
            cpp_s = TSector()
            for obj_dist, obj_type in s:
                cpp_s.Objects.push_back(pair[float, int](obj_dist, OBJ_TYPE_TO_NUM[obj_type]))
            self.diagram.Sectors.push_back(cpp_s)
        
    def dist_to(self, another_diagram):
        assert isinstance(another_diagram, Diagram)
        return DiagramDistance(self.diagram, (<Diagram>another_diagram).diagram)
    
    def __repr__(self):
        a = []
        for s in self.diagram.Sectors:
            a.append([(o.first, OBJ_TYPES[o.second]) for o in s.Objects])
        return repr(a)

 

In [4]:
%%script bash -c "cat > setup.py"
from distutils.core import setup
from Cython.Build import cythonize
import os

os.environ['CFLAGS'] = '-O3 -Wall -std=c++17'

setup(
    ext_modules = cythonize("diagram.pyx")
)

In [5]:
!python3 ./setup.py build_ext --inplace 

Compiling diagram.pyx because it changed.
[1/1] Cythonizing diagram.pyx
running build_ext
building 'diagram' extension
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -Wstrict-prototypes -O3 -Wall -std=c++17 -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.5m -c diagram.cpp -o build/temp.linux-x86_64-3.5/diagram.o
cc1plus: warning: command line option ‘-Wstrict-prototypes’ is valid for C/ObjC but not for C++
x86_64-linux-gnu-g++ -pthread -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -Wl,-z,relro -Wl,-Bsymbolic-functions -Wl,-z,relro -O3 -Wall -std=c++17 -Wdate-time -D_FORTIFY_SOURCE=2 build/temp.linux-x86_64-3.5/diagram.o -o /media/ura/Windows/Users/ura-lenovo/ubuntu_mount/Diploma/diagram.cpython-35m-x86_64-linux-gnu.so


In [6]:
from diagram import Diagram


In [7]:
help(Diagram)

Help on class Diagram in module diagram:

class Diagram(builtins.object)
 |  Optimized for fast dist_to operation
 |  Constructing is relatively slow operation
 |  
 |  Methods defined here:
 |  
 |  __init__(...)
 |      Example: 'Diagram([[(1, "water")], [(1, "forest"), (2, "water")]])'
 |      Support types: OBJ_TYPES
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and return a new object.  See help(type) for accurate signature.
 |  
 |  __reduce__ = __reduce_cython__(...)
 |  
 |  __repr__(self, /)
 |      Return repr(self).
 |  
 |  __setstate__ = __setstate_cython__(...)
 |  
 |  dist_to(...)
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  __pyx_vtable__ = <capsule object NULL>



In [8]:
Diagram([[(1, "water")], [(1, "wood"), (2, "water")]])

[[(1.0, 'water')], [(1.0, 'wood'), (2.0, 'water')]]

In [9]:
assert Diagram([[(1, "water")]]).dist_to(Diagram([[(1, "water")]])) == 0.0, "Incorrect equal"
assert Diagram([[(1, "water")]]).dist_to(Diagram([[(1, "wood")]])) == 1.0, "Incorrect different"
assert Diagram([[(1, "water")], [(1, "wood"), (2, "water")]]).dist_to(
    Diagram([[(1, "wood")], [(1, "water")]])) == 0.25, "Incorrect shift"


In [10]:
assert Diagram([[]]).dist_to(Diagram([[]])) == 0.0, "Incorrect shift"